# Donkey Colab with GPU(TensorFlow 2.0.0 Alpha0)

Donkey Colab with GPU.

- Version 0.1(2019年3月18日)

# TensorFlowを2.0.0 Alpha0にUpdate


In [0]:
!pip install tensorflow-gpu==2.0.0-Alpha0

# GPUの割り振りを確認

"Found GPU at: /device:GPU:0"と表示されれば、GPUが使用できる状態です。

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# DonkeyレポジトリをGit clone

In [0]:
!git clone -b dev https://github.com/autorope/donkeycar.git donkey

# インストール

In [0]:
!pip3 install -e donkey

# Projectの作成

In [0]:
!donkey createcar --template donkey2  /content/mycar

using donkey version: 2.5.7 ...
Creating car folder: /content/mycar
making dir  /content/mycar
Creating data & model folders.
making dir  /content/mycar/models
making dir  /content/mycar/data
making dir  /content/mycar/logs
Copying car application template: donkey2
Copying car config defaults. Adjust these before starting your car.
Donkey setup complete.


# datas.zipのアップロードとデータの解凍
下記コードを実行すると、ファイルのアップロード フォームが出現します。datas.zipをアップリードします。

datas.zipは、/content以下にアップされます。mv コマンドで、/content/mycar/dataに移動し、解凍します。

In [0]:
import os
from google.colab import files

if(os.path.exists("/content/datas.zip")):
   os.remove("/content/datas.zip")
if(os.path.exists("/content/mycar/data/datas.zip")):
   os.remove("/content/mycar/data/datas.zip")
   
uploaded = files.upload()

WORK_FOLDER = "/content/mycar/data/"
if(os.path.exists(WORK_FOLDER) == False):
  os.makedirs(WORK_FOLDER)

!mv /content/datas.zip /content/mycar/data/
%cd /content/mycar/data/
!unzip -o datas.zip

# ランタイムの再起動

Colabメニューの**[ランタイム]-[ランタイムの再起動]**して、ライブラリを取り込みます。

# 学習用のScript

DonkeyCarの学習用のScriptを読み込みます。

In [0]:
#!/usr/bin/env python3

import os

from docopt import docopt
import donkeycar as dk
import tensorflow as tf
from donkeycar.parts.camera import PiCamera
from donkeycar.parts.transform import Lambda
from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.actuator import PCA9685, PWMSteering, PWMThrottle
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.web_controller import LocalWebController
from donkeycar.parts.clock import Timestamp
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.transform import Lambda


def train(cfg, tub_names, new_model_path, base_model_path=None):
    """
    use the specified data in tub_names to train an artifical neural network
    saves the output trained model as model_name
    """
    X_keys = ['cam/image_array']
    y_keys = ['user/angle', 'user/throttle']

    new_model_path = os.path.expanduser(new_model_path)

    kl = KerasLinear()
    if base_model_path is not None:
        base_model_path = os.path.expanduser(base_model_path)
        kl.load(base_model_path)

    print('tub_names', tub_names)
    if not tub_names:
        tub_names = os.path.join("/content/mycar/data/", '*')
    print('tub_names', tub_names)
    tubgroup = TubGroup(tub_names)
    train_gen, val_gen = tubgroup.get_train_val_gen(X_keys, y_keys,
                                                    batch_size=128,
                                                    train_frac=0.8)

    total_records = len(tubgroup.df)
    total_train = int(total_records * 0.8)
    total_val = total_records - total_train
    print('train: %d, validation: %d' % (total_train, total_val))
    steps_per_epoch = total_train //128
    print('steps_per_epoch', steps_per_epoch)

    kl.train(train_gen,
             val_gen,
             saved_model_path=new_model_path,
             steps=steps_per_epoch,
             train_split=0.8)

#学習の開始

/conetnt/mycar/data/に存在するフォルダをすべて学習対象にする場合

In [0]:
with tf.device('/gpu:0'):
  cfg = None
  tub = None
  new_model_path = "/content/mycar/models/mypilot"
  base_model_path = None
  cache = None
  train(cfg, tub, new_model_path, base_model_path)

学習結果は、/content/mycar/models/mypilotに保存されます。
mypilotをダブルクリックしローカルにダウンロードし、SSHでRaspberryPiの~/mycar/models/mypilotにコピーします。

![](https://faboplatform.github.io/DonkeyDocs/1.DonkeyCar%E6%A7%8B%E7%AF%89%E3%83%BB%E5%AD%A6%E7%BF%92/img/upload.png)

#学習の開始

/conetnt/mycar/data/に存在する特定のフォルダを学習対象にする場合。

tubにフォルダの場所をフルパスで記載する。

In [0]:
with tf.device('/gpu:0'):
  cfg = None
  tub = "/content/mycar/data/tub_01-03-31"
  new_model_path = "/content/mycar/models/mypilot"
  base_model_path = None
  cache = None
  train(cfg, tub, new_model_path, base_model_path)